<h2><font color="#004D7F" size=5>Módulo 3: Boosting</font></h2>


<h1><font color="#004D7F" size=6> 3. Extreme Gradient Boosting</font></h1>
<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Aprendizaje Automático II</font><br>
<font color="#004D7F" size=3>Universidad Nacional de Educación a Distancia</font>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. Algoritmo XGBoost](#section1)
    * [1.1. Instalar XGBoost](#section11)
* [2. XGBoost según el tipo de problema](#section2)
    * [2.1. XGBoost para Clasificación](#section21)
    * [2.2. XGBoost para Regresión](#section22)
* [3. Hiperparámetros de XGBoost](#section3)
   * [3.1. Número de árboles](#section31)
   * [3.2. Profundidad del árbol](#section32)
   * [3.3. Tasa de aprendizaje](#section33)
   * [3.4. Tamaño de la muestra](#section34)
   * [3.5. Número de características](#section35)
* [Ejercicios](#sectionEj)

---

<a id="section0"></a>
# <font color="#004D7F">0. Contexto</font>

Extreme Gradient Boosting (XGBoost) es una biblioteca de código abierto que proporciona una implementación eficiente y efectiva del algoritmo. En este tutorial, trabajaremos:
- Extreme Gradient Boosting es una implementación eficiente de código abierto del algoritmo GBM.
- Cómo desarrollar conjuntos XGBoost para clasificación y regresión con la API scikit-learn.
- Cómo explorar el efecto de los hiperparámetros del modelo XGBoost en el rendimiento del modelo.


---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Algoritmo Extreme Gradient Boosting</font>

- Extreme Gradient Boosting (XGBoost) es una implementación eficiente de código abierto del algoritmo GBM.
- Fue desarrollado inicialmente por Tianqi Chen y descrito por Chen y Carlos Guestrin en su artículo de 2016 titulado _"XGBoost: A Scalable Tree Boosting System"_.
- Las dos razones principales para utilizar XGBoost son la velocidad de ejecución y el rendimiento del modelo.
  
<figure><center>
  <img src="data/xgboost.jpg" width="450" height="350" alt="Gráfica">
  <figcaption><blockquote>XGBoost. Extraída de <a href="http://dx.doi.org/10.1007/s42107-023-00651-z">Hybrid machine learning approach for construction cost estimation: an evaluation of extreme gradient boosting model</a></blockquote></figcaption>
</center></figure>


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Acceso al artículo científico [_XGBoost: A Scalable Tree Boosting System_](https://dl.acm.org/doi/abs/10.1145/2939672.2939785)
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Acceso al artìculo de que demuestra empíricamente la alta eficiencia de XGBoost llamado [_Benchmarking Random Forest Implementations_](https://www.r-bloggers.com/2015/05/benchmarking-random-forest-implementations/)
</div>

<a id="section11"></a> 
## <font color="#004D7F"> 1.1. Instalar XGBoost</font>

El primer paso es instalar la biblioteca XGBoost.

In [ ]:
!pip install xgboost

Luego puede confirmar que la biblioteca XGBoost se instaló correctamente y se puede utilizar ejecutando el siguiente script.

In [ ]:
import xgboost
print(xgboost.__version__)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre la librería [`XGBoost`](https://xgboost.readthedocs.io/en/stable/)
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre la librería [`XGBoost en Python`](https://xgboost.readthedocs.io/en/stable/python/index.html)
</div>

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section2"></a> 
# <font color="#004D7F"> 2. XGBoost según el tipo de problema</font>

La biblioteca XGBoost tiene su propia API personalizada, aunque usaremos el método a través de las clases contenedoras de scikit-learn: `XGBRegressor` y `XGBClassifier`. 

Al ajustar un modelo final, puede ser deseable aumentar el número de árboles hasta que la varianza del modelo se reduzca en evaluaciones repetidas, o ajustar múltiples modelos finales y promediar sus predicciones. 

<a id="section21"></a> 
## <font color="#004D7F"> 2.1. XGBoost para Clasificación</font>

En esta sección, veremos el uso de XGBoost para un problema de clasificación.

<a id="section211"></a> 
### <font color="#004D7F"> 2.1.1. Dataset</font>

Primero, podemos usar la función `make_classification()` para crear un problema de clasificación binaria sintética con 1000 ejemplos y 20 características de entrada.

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=3)
print(X.shape, y.shape)

<a id="section212"></a> 
### <font color="#004D7F"> 2.1.2. Evaluación</font>

Evaluaremos el modelo utilizando una validación cruzada estratificada repetida de _k_ veces, con 3 repeticiones y 10 veces. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre la clase [`XGBClassifier`](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier)
</div>

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

model = XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section22"></a> 
## <font color="#004D7F"> 2.2. XGBoost para Regresión</font>

En esta sección, veremos el uso de XGBoost para un problema de regresión. 

<a id="section221"></a> 
### <font color="#004D7F"> 2.2.1. Dataset</font>

Primero, podemos usar la función `make_regression()` para crear un problema de regresión sintética con 1000 ejemplos y 20 características de entrada.

In [ ]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=2)
print(X.shape, y.shape)

<a id="section222"></a> 
### <font color="#004D7F"> 2.2.2. Evaluación</font>

Evaluaremos el modelo mediante validación cruzada estratificada repetida de _k_ veces, con 3 repeticiones y 10 pliegues. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre la clase [`XGBRegressor`](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)
</div>

In [ ]:
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

model = XGBRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: La API de scikit-learn invierte el signo del MAE para transformarlo, de minimizar el error a maximizar el error negativo. Esto significa que los errores positivos de gran magnitud se convierten en grandes errores negativos (por ejemplo, 100 se convierte en -100) y un modelo perfecto no tiene ningún error con un valor de 0,0. También significa que podemos ignorar con seguridad el signo de las puntuaciones MAE medias. 

</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section3"></a> 
# <font color="#004D7F"> 3. Hiperparámetros de XGBoost</font>

En esta sección, analizaremos más de cerca algunos de los hiperparámetros que debería considerar ajustar para el conjunto XGBoost y su efecto en el rendimiento del modelo.

<a id="section31"></a> 
## <font color="#004D7F"> 3.1. Número de árboles</font>

- Los árboles de decisión se agregan al modelo secuencialmente en un esfuerzo por corregir y mejorar las predicciones hechas por árboles anteriores.
- Por eso, cuantos más árboles haya, mejor será.
- El número de árboles se establece con el argumento `n_estimators` (por defecto es 100). 

In [ ]:
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
	return X, y

def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

In [ ]:
def get_models():
	models = ???
	trees = ???
	???
		models[str(n)] = ???
	return models

In [ ]:
from matplotlib import pyplot

X, y = get_dataset()
models = get_models()
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section32"></a> 
## <font color="#004D7F"> 3.2. Profundidad del árbol</font>

- La profundidad del árbol controla qué tan especializado está cada árbol en train (qué tan general o sobreajustado).
- Se prefieren árboles que no sean demasiado superficiales y generales (como AdaBoost) ni demasiado profundos y especializados (como Bootstrap Aggregation).
- XGBoost generalmente funciona bien con árboles que tienen una profundidad modesta, encontrando un equilibrio entre habilidad y generalidad. 
- La profundidad del árbol se controla mediante el argumento `max_depth` (por defecto es 6).

In [ ]:
def get_models():
	models = dict()
	# Explorar entre 1 y 10
	???
		models[str(i)] = ???
	return models

In [ ]:
X, y = get_dataset()
models = get_models()
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section33"></a> 
## <font color="#004D7F"> 3.3. Tasa de aprendizaje</font>

- La tasa de aprendizaje controla la cantidad de contribución que cada modelo tiene en la predicción del conjunto.
- Tasas más pequeñas pueden requerir más árboles de decisión en el conjunto. 
- La tasa de aprendizaje se puede controlar mediante el argumento `eta` (por defecto es 0,3).

In [ ]:
def get_models():
	models = dict()
	# explorar estas tasas de aprendizaje [0.0001, 0.001, 0.01, 0.1, 1.0]
	???
	???
		key = '%.4f' % r
		models[key] = ???
	return models

In [ ]:
X, y = get_dataset()
models = get_models()
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section34"></a> 
## <font color="#004D7F"> 3.4. Tamaño de la muestra</font>

- Usar menos instancias introduce más varianza para cada árbol, aunque puede mejorar el rendimiento general del modelo. 
- Se especifica mediante el argumento `subsample` (por defecto es 1.0). 

In [ ]:
from numpy import arange

def get_models():
	models = dict()
	# explorar de 10 a 100% en rangos de 10&
	???
		key = '%.1f' % i
		models[key] = ???
	return models

In [ ]:
X, y = get_dataset()
models = get_models()
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section35"></a> 
## <font color="#004D7F"> 3.5. Número de características</font>

- La cantidad de características introduce una varianza adicional en el modelo, lo que puede mejorar el rendimiento, aunque podría requerir un aumento en la cantidad de árboles. 
- Se especifica mediante el argumento `colsample_bytree` (por defecto es 1,0). 

In [ ]:
def get_models():
	models = dict()
	# explore ratio of features from 10% to 100% in 10% increments
	???
		key = '%.1f' % i
		models[key] = ???
	return models

In [ ]:
X, y = get_dataset()
models = get_models()
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="sectionEj"></a>
<h3><font color="#004D7F" size=6> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicios</font></h3>

Se proponen las siguientes actividades para consolidar el aprendizaje.

# <font color="#004D7F" size=5>Ejercicio 1</font>
__Hiperparámetros__. Explore diferentes configuraciones de hiperparámetros que veas en la librería sobre XGBoost y comente los resultados. 

# <font color="#004D7F" size=5>Ejercicio 2</font>
__Problema de Regresión__. XGBoost se puede utilizar con árboles de regresión. En lugar de predecir el valor de clase más común del conjunto de predicciones, puede devolver el promedio de las predicciones. Experimente con problemas de regresión.

# <font color="#004D7F" size=5>Ejercicio 3</font>
__Datasets reales__. Busque un dataset original y verdadero (que no sea sintético) y evalúe el uso de los conceptos vistos en esta unidad. Los conjuntos de datos en pueden ser obtenidos del [repositorio de aprendizaje automático de UCI](https://archive.ics.uci.edu/).

# <font color="#004D7F" size=5>Ejercicio 4</font>
__Hiperparámetro__. XGBoost también puede tomar muestras de columnas para cada división, y esto está controlado por el argumento `colsample_bylevel`. Explora el impacto de este hiperparámetro.

# <font color="#004D7F" size=5>Ejercicio 5</font>
__Búsqueda de la mejor configuración__. Como se ha visto existen diferentes hiperparámetros que pueden ajustar nuestro modelo. Haga una búsqueda para un dataset real de cuales, entre un rango amplio de hiperparétros, maximizan la métrica. Puede utilizar una búsqueda aleatoria.

---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>